# Домашнее задание №2 "Классификация текстов" (модуль "Обработка естественного языка")

## Задание: Сделать классификацию данных fakenews

Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

## Библиотеки

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
import random

In [ ]:
import re

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install pymorphy2

import pymorphy2
m = pymorphy2.MorphAnalyzer()

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from collections import Counter

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
import warnings
warnings.filterwarnings('ignore')


## Данные

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2024-09-05 21:02:56--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv.1’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.06s   

2024-09-05 21:02:57 (18.5 MB/s) - ‘Constraint_Train.csv.1’ saved [1253562/1253562]



In [ ]:
df = pd.read_csv('Constraint_Train.csv')

In [ ]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


### Первый способ: усреднение ембеддингов слов в тексте

In [ ]:
# Функции предобработки текста

mystopwords = stopwords.words('english')

# оставляем только буквы
eng_words = re.compile("[a-z]+")

def words_only(text):
    return " ".join(eng_words.findall(text))


def lemmatize(text, mystem=m):

    # strip убирает пробелы в начале и конце текста
    try:
        return " ".join([m.parse(w)[0].normal_form for w in text.split(' ')]).strip()
    except:
        return " "

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

# Удаление https:
def del_http(text):
    return re.sub(r'http\S+', '', text)

def preprocess(text):

    return text
    #return text.lower()
    #return del_http(text.lower())
    #return words_only(text.lower())
    #return lemmatize(words_only(del_http(text).lower()))


In [ ]:
n = 6
preprocess(df['tweet'][n]), df['tweet'][n]

('If you tested positive for #COVID19 and have no symptoms stay home and away from other people. Learn more about CDC’s recommendations about when you can be around others after COVID-19 infection: https://t.co/z5kkXpqkYb. https://t.co/9PaMy0Rxaf',
 'If you tested positive for #COVID19 and have no symptoms stay home and away from other people. Learn more about CDC’s recommendations about when you can be around others after COVID-19 infection: https://t.co/z5kkXpqkYb. https://t.co/9PaMy0Rxaf')

In [ ]:
# Предпроцессинг и токенизация

sentences_v1 = [word_tokenize(preprocess(text)) for text in tqdm(df.tweet)]




  0%|          | 0/6420 [01:22<?, ?it/s]



  4%|▍         | 271/6420 [00:00<00:02, 2458.40it/s]


  9%|▉         | 608/6420 [00:00<00:01, 2969.36it/s]


 14%|█▍        | 908/6420 [00:00<00:01, 2860.34it/s]


 19%|█▉        | 1209/6420 [00:00<00:01, 2914.63it/s]


 24%|██▍       | 1544/6420 [00:00<00:01, 3066.02it/s]


 29%|██▉       | 1854/6420 [00:00<00:01, 3076.92it/s]


 34%|███▎      | 2166/6420 [00:00<00:01, 3090.57it/s]


 39%|███▊      | 2476/6420 [00:00<00:01, 2998.90it/s]


 44%|████▍     | 2814/6420 [00:00<00:01, 3112.12it/s]


 49%|████▉     | 3133/6420 [00:01<00:01, 3132.87it/s]


 54%|█████▍    | 3467/6420 [00:01<00:00, 3194.92it/s]


 59%|█████▉    | 3793/6420 [00:01<00:00, 3213.92it/s]


 64%|██████▍   | 4115/6420 [00:01<00:00, 2934.68it/s]


 69%|██████▉   | 4414/6420 [00:01<00:00, 2883.50it/s]


 73%|███████▎  | 4715/6420 [00:01<00:00, 2917.89it/s]


 78%|███████▊  | 5034/6420 [00:01<00:00, 2994.34it/s]


 84%|████████▎ | 5363/6420 [00:01<00:00, 3076.64it/s]


 88%

In [ ]:
# задаем модель Word2Vec для векторизации текстов

%time model_v1 = Word2Vec(sentences_v1, workers=4, vector_size=300, min_count=3, window=7, epochs=15)

CPU times: user 9.77 s, sys: 40 ms, total: 9.81 s
Wall time: 6.16 s


In [ ]:
print(type(model_v1))

<class 'gensim.models.word2vec.Word2Vec'>


In [ ]:
model_v1.wv.most_similar('success')

[('version', 0.7264477610588074),
 ('f', 0.7140642404556274),
 ('moment', 0.7137652635574341),
 ('giving', 0.710201621055603),
 ('crown', 0.7006908059120178),
 ('morning', 0.699194610118866),
 ('tabligi', 0.6981774568557739),
 ('allocation', 0.6971296668052673),
 ('investment', 0.6955597400665283),
 ('adding', 0.6850079894065857)]

In [ ]:
# эмбеддинг текста - усредненный эмбеддинг слов, входящих в него

def get_text_embedding_v1(text, model_w2v, vector_size):

    result = []
    for word in word_tokenize(preprocess(text)):

        if word in model_w2v.wv:
            result.append(model_w2v.wv[word])

    if len(result):
        result = np.average(result, axis=0)
    else:
        result = np.zeros(vector_size)
    return result

In [ ]:
features_v1 = [get_text_embedding_v1(text, model_v1, 300) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:03<00:00, 2095.76it/s]


In [ ]:
# Классификация текстов для первой модели Word2Vec: vector_size=300, min_count=3, window=7

features_v1 = [get_text_embedding_v1(text, model_v1, 300) for text in tqdm(df.tweet)]
X_train, X_test, y_train, y_test = train_test_split(features_v1, df.label, test_size=0.33)

model_l1 = LogisticRegression()
model_l1.fit(X_train, y_train)

predicted_v1 = model_l1.predict(X_test)

report = classification_report(y_test, predicted_v1, output_dict=True)

macro_precision =  report['macro avg']['precision']
macro_recall = report['macro avg']['recall']
macro_f1 = report['macro avg']['f1-score']

print('F1:', macro_f1)

100%|██████████| 6420/6420 [00:03<00:00, 2091.98it/s]


F1: 0.9072490307826988


#### Оптимальные параметры модели Word2Vec (vector_size, min_count, window)

In [ ]:
def optimize_word2vec(vector_size, min_count, window):

    model_tmp = Word2Vec(sentences_v1, workers=4, vector_size=vector_size, min_count=min_count, window=window, epochs=15)

    features_v1 = [get_text_embedding_v1(text, model_tmp, vector_size) for text in df.tweet]

    X_train, X_test, y_train, y_test = train_test_split(features_v1, df.label, test_size=0.33)

    model_l1 = LogisticRegression()
    model_l1.fit(X_train, y_train)

    predicted_v1 = model_l1.predict(X_test)

    report = classification_report(y_test, predicted_v1, output_dict=True)

    macro_precision =  report['macro avg']['precision']
    macro_recall = report['macro avg']['recall']
    macro_f1 = report['macro avg']['f1-score']

    return macro_f1


In [ ]:
# Поиск лучших параметров модели Word2Vec по метрике f1_macro

random.seed(42)
outputs = []
vector_sizes = [256, 300, 512]
min_counts = [3, 6, 9]
windows = [10, 15, 20]

for vsize_ in tqdm(vector_sizes):

  print()
  print(f'vector_size: {vsize_}')

  for min_count_ in min_counts:
    for window_ in windows:

        #print(f'vsize: {vsize_}, min_count: {min_count_}, window: {window_}')
        f1_score = optimize_word2vec(vsize_, min_count_, window_ )

        el = [vsize_, min_count_, window_, f1_score]
        outputs.append(el)

result_df = pd.DataFrame(outputs, columns=['vector_size', 'min_count', 'window', 'f1_score'])

result_df.sort_values(by='f1_score', ascending=False)


  0%|          | 0/3 [00:00<?, ?it/s]


vector_size: 256


 33%|███▎      | 1/3 [01:16<02:32, 76.19s/it]


vector_size: 300


 67%|██████▋   | 2/3 [02:42<01:22, 82.40s/it]


vector_size: 512


100%|██████████| 3/3 [04:24<00:00, 88.16s/it]


,vector_size,min_count,window,f1_score
1,256,3,15,0.919283
22,512,6,15,0.917531
11,300,3,20,0.917139
26,512,9,20,0.914942
20,512,3,20,0.914728
16,300,9,15,0.913503
21,512,6,10,0.913414
23,512,6,20,0.909693
15,300,9,10,0.909685
2,256,3,20,0.909301


Интересный вывод: если проводить "глубокую" предобработку текста
- текст приводится к нижнему регистру
- удаляются ссылки на статьи (начинаются на https:)
- выбираются только слова
- убираются стоп-слова
- слова лемматизируются
то результат составляет 88% по f1- мере

Если предобработку не проводить, то точность выше 91%, что означает, что значимые признаки находятся не только в словах, но и в каких-то символах, влияющих на класс обьекта

### Второй способ: эмбеддинг текста через CountVectorizer (частотность слов)

In [ ]:
vec_v2 = CountVectorizer()

In [ ]:
bow_v2 = vec_v2.fit_transform(df.tweet)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow_v2, df.label, test_size=0.33)
model_v2 = LogisticRegression()
model_v2.fit(X_train, y_train)

LogisticRegression()

In [ ]:
predicted_v2 = model_v2.predict(X_test)
print(classification_report(y_test, predicted_v2))


              precision    recall  f1-score   support

        fake       0.91      0.93      0.92      1004
        real       0.93      0.92      0.93      1115

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



### Третий способ PyTorch + LSTM

In [ ]:
# Предпроцессинг и токенизация

sentences_v3 = [word_tokenize(preprocess(text)) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:03<00:00, 1871.19it/s]


In [ ]:
# задаем модель Word2Vec для векторизации слов

%time model_v3 = Word2Vec(sentences_v3, workers=4, vector_size=256, min_count=3, window=15, epochs=15)

CPU times: user 9.57 s, sys: 73.1 ms, total: 9.65 s
Wall time: 5.19 s


In [ ]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для максимальной длины предложений.

In [ ]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [ ]:
# Всего слов в первом твите
len(token_lists[0])

32

In [ ]:
# Всего твитов
len(token_lists)

6420

In [ ]:
# Самый длинный твит ---> 1592 слова
max_len

1592

In [ ]:
# Количество слов в твите

fd_v3 = Counter([len(tokens) for tokens in token_lists])

In [ ]:
# Количество слов в предложении ---> частота
fd_v3.most_common(10)

[(20, 178),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 168),
 (21, 168),
 (16, 163),
 (17, 162),
 (15, 160),
 (23, 156)]

Возьмём те же w2v эмбеддинги.

In [ ]:
# Формируем матрицу признаков для твита
# по строкам размерность длина твита в словах =  N_words
# по столбцам эмбеддинг слова размерности 256
# 1 твит ----> 1 метка ----> features (50 * 256)

def get_word_embedding_v3(tokens, max_len):
    result = []

    # Проходим по всем словам твита с индексами от 0 до max_len
    for i in range(max_len):

        # Если слов не осталось, то до max_len дополняем нулевыми
        if i < len(tokens):

            # Берем слово из твита
            word = tokens[i]

            # Если эмбеддинг слова есть в модели, то берем его
            if word in model_v3.wv:
                result.append(model_v3.wv[word])
            else:
                result.append(np.zeros(256))
        else:
            # Дополняем нулями, если слов в твите меньше max_len = 200
            result.append(np.zeros(256))
    return result

In [ ]:
# Во внимание принимаем только N_words первых слов твита
N_words = 40

features_v3 = [get_word_embedding_v3(text, N_words) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:00<00:00, 9203.44it/s]


In [ ]:
class Net_v3(nn.Module):

    def __init__(self):
        super(Net_v3, self).__init__()

        # Эмбеддинги размера 256
        # Скрытый слой задаем сами  = 64

        self.lstm = nn.LSTM(256, 52)
        self.out = nn.Linear(52, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net_v3()
#net.cuda()
net.cpu()
print(net)

Net_v3(
  (lstm): LSTM(256, 52)
  (out): Linear(in_features=52, out_features=1, bias=True)
)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_v3, labels, test_size=0.25)

In [ ]:
# ---> 1мин
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
in_data.shape

torch.Size([4815, 40, 256])

In [ ]:
# cuda - видеокарта
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)

cpu


Net_v3(
  (lstm): LSTM(256, 52)
  (out): Linear(in_features=52, out_features=1, bias=True)
)

In [ ]:
device

device(type='cpu')

In [ ]:
#optimizer = optim.Adam(net.parameters(), lr=0.01)
#criterion = nn.BCEWithLogitsLoss()

optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        #batch_x = in_data[i:i + batch_size].cuda()
        #batch_y = targets[i:i + batch_size].cuda()
        batch_x = in_data[i:i + batch_size] .cpu()
        batch_y = targets[i:i + batch_size] .cpu()
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
net.train()
train_one_epoch(in_data, targets)

100%|██████████| 301/301 [00:02<00:00, 110.86it/s]

tensor(0.5723, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
train_one_epoch(in_data, targets)


100%|██████████| 301/301 [00:02<00:00, 112.36it/s]

tensor(0.5035, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
for i in range(10):
  if i% 10 == 0:
    print('Эпоха', i)
  train_one_epoch(in_data, targets)

Эпоха 0


100%|██████████| 301/301 [00:03<00:00, 79.20it/s] 


tensor(0.0144, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 301/301 [00:02<00:00, 110.99it/s]


tensor(0.0162, grad_fn=<BinaryCrossEntropyBackward0>)


...

tensor(0.0136, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 301/301 [00:02<00:00, 108.23it/s]


tensor(0.0128, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 301/301 [00:03<00:00, 78.23it/s] 

tensor(0.0126, grad_fn=<BinaryCrossEntropyBackward0>)


Что получилось?

In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test.cpu()).reshape(-1)

In [ ]:
result = (output.cpu() > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

0.9339563862928348



---



### Четвертый способ PyTorch без LSTM

In [ ]:
# Предпроцессинг и токенизация

sentences_v4 = [word_tokenize(preprocess(text)) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:02<00:00, 2772.67it/s]


In [ ]:
# задаем модель Word2Vec для векторизации текстов

model_v4 = Word2Vec(sentences_v4, workers=4, vector_size=256, min_count=5, window=15, epochs=15)

In [ ]:
features_v4 = [get_text_embedding_v1(text, model_v4, 256) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:03<00:00, 1915.59it/s]


In [ ]:
class Net_v4(nn.Module):

    def __init__(self):
        super(Net_v4, self).__init__()
        self.out = nn.Linear(256, 1)

    def forward(self, x):
        return torch.sigmoid(self.out(x))


net = Net_v4()
#net.cuda()
net.cpu()
print(net)

Net_v4(
  (out): Linear(in_features=256, out_features=1, bias=True)
)


In [ ]:
#optimizer = optim.Adam(net.parameters(), lr=0.01)
#criterion = nn.BCEWithLogitsLoss()

optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCELoss()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_v4, labels, test_size=0.25)

In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
def train_one_epoch_v4(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size].cpu()
        batch_y = targets[i:i + batch_size].cpu()
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
for i in range(10):
  train_one_epoch_v4(in_data, targets)

100%|██████████| 301/301 [00:00<00:00, 1541.66it/s]


tensor(0.2381, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 301/301 [00:00<00:00, 1622.48it/s]


tensor(0.2381, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 301/301 [00:00<00:00, 1681.53it/s]


...

tensor(0.2381, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 301/301 [00:00<00:00, 1344.86it/s]


tensor(0.2381, grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 301/301 [00:00<00:00, 1383.46it/s]

tensor(0.2381, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test.cpu()).squeeze(1)

In [ ]:
result = (output.cpu() > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

0.907165109034268

# Итоги

In [ ]:
# Сравнительная таблица показателей моделей

status = ['модель','предобработка текста','Word2Vec - параметры', 'параметры нейросети', 'f1_score']

M1 = np.array(['Word2Vec усреднение ембедингов слов в тексте', 'text.lower()', \
               'Word2Vec:vector_size = 256, min_count = 3, window = 15',\
               ' - ', '0.919'])

M2 = np.array(['CountVectorizer', 'без предобработки', ' - ', ' - ' , '0.92'])

M3 = np.array(['Нейросеть PyTorch + LSTM', \
               'text.lower()', \
               'Word2Vec:vector_size = 256, min_count = 3, window = 15', \
                'LSTM - 256*52, Linear = 52 + sigmoid,  max_len_text = 40 ',\
                '0.933'])

M4 = np.array(['Нейросеть PyTorch без LSTM', \
               'text.lower()', \
               'Word2Vec:vector_size = 256, min_count = 3, window = 15',\
               'Linear 256 + sigmoid',\
               '0.907'])

T = np.vstack([M1, M2, M3, M4])

result = pd.DataFrame(data = T, columns= status)
result

,модель,предобработка текста,Word2Vec - параметры,параметры нейросети,f1_score
0,Word2Vec усреднение ембедингов слов в тексте,text.lower(),"Word2Vec:vector_size = 256, min_count = 3, win...",-,0.919
1,CountVectorizer,без предобработки,-,-,0.92
2,Нейросеть PyTorch + LSTM,text.lower(),"Word2Vec:vector_size = 256, min_count = 3, win...","LSTM - 256*52, Linear = 52 + sigmoid, max_len...",0.933
3,Нейросеть PyTorch без LSTM,text.lower(),"Word2Vec:vector_size = 256, min_count = 3, win...",Linear 256 + sigmoid,0.907


Для задачи классификации текстов на 2 класса новостей (fake, real) рассмотрены различные модели.

1.На этапе предпроцессинга текста не удалось найти такое преобразование текста, которое повысило бы точность модели (были проверены операции удаления ссылок на адрес сайта, лемматизация, фильтрация только слов, исключение цифр, знаков пунктуации и стоп слов)

2.Для формирования контекстного векторного представления слов использовалась модель Word2Vec. По сетке параметров были найдены их наилучшие значения. Наибольший вклад в улучшение модели внес размер контекстного окна (window = 15).

3.При обучении нейросетевых моделей лучший результат показала модель 2, содержащая слой LSTM (f1_score = 0.933)
  




